<a href="https://colab.research.google.com/github/Santosh-Gupta/NaturalLanguageRecommendations/blob/master/medical_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!pip install langid
!pip install tqdm

     |████████████████████████████████| 1.9MB 4.6MB/s 
  Created wheel for langid: filename=langid-1.1.6-cp36-none-any.whl size=1941190 sha256=699bf2d6b835147f8fff54168cb6831f3786472bf5c66de0e144b1ea52b4847f
  Stored in directory: /root/.cache/pip/wheels/29/bc/61/50a93be85d1afe9436c3dc61f38da8ad7b637a38af4824e86e
Successfully built langid


In [0]:
import zipfile
import os
import sys
import pandas as pd
import numpy as np
import gc
from urllib import request
import json
import itertools
import gzip
import shutil
import ast
import pickle 
import nltk
from langid.langid import LanguageIdentifier, model
from tqdm import tqdm

In [0]:
# Function to unip a file from semantic scholar to a json file, then open that json file
def dlandopen(url, file, json ):
    request.urlretrieve(url, file )
    with gzip.open(file, 'rb') as f_in:
        with open(json, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

    os.remove(file)
    f = open(json)
    mylistH = f.readlines()
    del f
    return mylistH

In [0]:
# This is probably not the best way to go about it, but this is the code to convert the opened file into a pandas dataframe
#Akul, it may be worth figuring out if you can make a dataframe directly, without going line-by-line 
def firstDataframe(mylist, start, finish):
    data = []

    for line in itertools.islice(mylist , start , finish):
       data.append(json.loads(line))

    df = pd.DataFrame(data)
    del data
    gc.collect()
    return df

In [0]:
def medicalRefinement(dftbr):
  dftbr['strsources'] = dftbr['sources'].astype('str')
  dftbr['totalCites2'] = dftbr['outCitations'] + dftbr['inCitations'] 
  dftbr = dftbr[dftbr['totalCites2'].str.len() >= 1]  #outCitations - all papers where papers cited is greater than 1
  dftbr = dftbr[ (dftbr.pmid != '' ) | (dftbr.strsources.str.contains("Medline"))]
  dftbr.drop(['journalVolume','journalPages','year','authors','sources','doiUrl', 'strsources'], axis = 1)
  return dftbr

In [0]:
def remove_nonenglish(df, jSet, vSet):
  # Gets list of English Journals
  listOfEnglishJournals = []
  identifier = LanguageIdentifier.from_modelstring(model, norm_probs=True)
  count = 0
  for i in jSet:
    lang = identifier.classify(str(i))
    langstr = str(lang)
    if langstr.find("en") != -1 and lang[1] > 0.5:
      listOfEnglishJournals.append(i)
  
  # Gets List of English Venues
  listOfEnglishVenues = []
  count = 0
  for i in vSet:
    lang = identifier.classify(str(i))
    langstr = str(lang)
    if langstr.find("en") != -1 and lang[1] > 0.5:
      listOfEnglishVenues.append(i)

  filter1 = df_refined['journalName'].isin(listOfEnglishJournals)
  filter2 = df_refined['venue'].isin(listOfEnglishVenues)

  dfFinal = df_refined[df_refined['journalName'].isin(listOfEnglishJournals) | df_refined['venue'].isin(listOfEnglishVenues)]

  return dfFinal

In [8]:
url_template = 'https://s3-us-west-2.amazonaws.com/ai2-s2-research-public/open-corpus/2019-11-01/s2-corpus-{:03}.gz'
file_template = 'json{:03}.gz'
json_template = 'json{:03}'

i=55
mylist = dlandopen(url_template.format(i), file_template.format(i), json_template.format(i) )
df = firstDataframe(mylist, 0, len(mylist))
df_refined = medicalRefinement(df)
print("REFINED")

KeyboardInterrupt: ignored

In [0]:
df_refined.shape

(112419, 21)

In [0]:
journalSet = set()
venueSet = set()
journalSet = journalSet|set(df_refined['journalName'])
venueSet = journalSet|set(df_refined['venue'])
english_only_df = remove_nonenglish(df_refined, journalSet, venueSet)
print("NONENGLISH REMOVED")

NONENGLISH REMOVED


In [0]:
english_only_df.shape

(76791, 21)

In [0]:
english_only_df.to_json("/content/gdrive/My Drive/text2doc_data/med_data/med-{:03}.json".format(i), orient='index')

In [8]:
url_template = 'https://s3-us-west-2.amazonaws.com/ai2-s2-research-public/open-corpus/2019-11-01/s2-corpus-{:03}.gz'
file_template = 'json{:03}.gz'
json_template = 'json{:03}'

all_length = 0
english_length = 0

i = 175

while i <= 178:
  mylist = dlandopen(url_template.format(i), file_template.format(i), json_template.format(i) )
  df = firstDataframe(mylist, 0, len(mylist))
  df_refined = medicalRefinement(df)
  df_refined.to_json("/content/gdrive/My Drive/text2doc_data/med_data_all/med-{:03}.json".format(i), orient='index')
  all_length += df_refined.shape[0]
  print("REFINED")
  journalSet = set()
  venueSet = set()
  journalSet = journalSet|set(df_refined['journalName'])
  venueSet = journalSet|set(df_refined['venue'])
  english_only_df = remove_nonenglish(df_refined, journalSet, venueSet)
  print("NONENGLISH REMOVED")
  english_length += english_only_df.shape[0]
  english_only_df.to_json("/content/gdrive/My Drive/text2doc_data/med_data_english_only/med-{:03}.json".format(i), orient='index')
  i+=1

print("All length: " + all_length)
print("English length: " + english_length)

REFINED
NONENGLISH REMOVED
REFINED
NONENGLISH REMOVED
REFINED
NONENGLISH REMOVED
REFINED
NONENGLISH REMOVED
REFINED
NONENGLISH REMOVED
REFINED
NONENGLISH REMOVED
REFINED
NONENGLISH REMOVED
REFINED
NONENGLISH REMOVED
REFINED
NONENGLISH REMOVED
REFINED
NONENGLISH REMOVED
REFINED
NONENGLISH REMOVED
REFINED
NONENGLISH REMOVED
REFINED
NONENGLISH REMOVED
REFINED
NONENGLISH REMOVED
REFINED
NONENGLISH REMOVED
REFINED
NONENGLISH REMOVED
REFINED
NONENGLISH REMOVED
REFINED
NONENGLISH REMOVED
REFINED
NONENGLISH REMOVED
REFINED
NONENGLISH REMOVED
REFINED
NONENGLISH REMOVED
REFINED
NONENGLISH REMOVED


OSError: ignored